In [0]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.distributions.categorical import Categorical
from torchsummary import summary
import matplotlib.pyplot as plt
import genattack_class as GA

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
## Neural Network Architecture for MNIST and similar datasets
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
## Neural Network Architecture for CIFAR-10
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def adv_train(model, device, train_loader, optimizer, epoch, epsilon=0.1, alpha=0.5):
    '''
    Implementation of Adversarial Training using FGSM
    '''
    log_interval = 120
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #print(data.shape)
        data.requires_grad = True
        
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward(retain_graph=True)

        # Generating Adversarial Examples
        # Collect gradient of inputs
        data_grad = data.grad.data

        # FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Get loss from perturbed data
        adv_output = model(perturbed_data)
        adv_loss = F.nll_loss(adv_output, target)


        optimizer.zero_grad()

        # Calculate total loss
        total_loss = alpha*loss + (1-alpha)*adv_loss
        total_loss.backward()

        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def train(model, device, train_loader, optimizer, epoch, epsilon=0.1, alpha=0.5):
    log_interval = 120
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() 
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def fgsm_attack(image, epsilon, data_grad):
    
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [0]:
def test(model, device, test_loader):
    '''
    Test model
    '''
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def adv_test(model, device, test_loader, epsilon=0.1):
    '''
    Tests model on FGSM adversarial examples
    '''
    model.eval()
    test_loss = 0
    correct = 0
    adv_examples = []
    for batch_data, batch_target in test_loader:
      for i in range(batch_data.shape[0]):
        data = batch_data[i].unsqueeze(0)
        target = batch_target[i].unsqueeze(0)
        data, target = data.to(device), target.to(device)
        data.requires_grad = True
        
        output = model(data)
        
        init_pred = output.max(1, keepdim=True)[1]
        
        #print(init_pred)
        if init_pred[0].item() != target.item():
            continue

        loss = F.nll_loss(output, target)
        model.zero_grad()

        loss.backward()

        data_grad = data.grad.data

        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1]
        if final_pred.item() == target.item():
            correct += 1
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
        
    final_acc = correct/float(len(test_loader))

    print("Epsilon: {}\tTest Accuracy = {} / {} = {}\n".format(epsilon, correct, len(test_loader), final_acc))
    return final_acc, adv_examples

In [0]:
def main():
    '''
    Trains model on dataset
    '''
    # Training settings
    batch_size = 100
    test_batch_size = 1000
    lr = 0.1 # learning rate
    #gamma = 0.7 # learning rate decay
    epochs = 20
    save_model = True

    use_cuda = torch.cuda.is_available()
    print("CUDA? " + str(use_cuda))

    torch.manual_seed(20294798)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST('../data' ,train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    model_adv = Net().to(device)

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    optimizer_adv = optim.SGD(model_adv.parameters(), lr=lr, momentum=0.9)

    #scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        print("Conventional Model")
        train(model, device, train_loader, optimizer, epoch)
        print("Adversarially trained Model")
        adv_train(model_adv, device, train_loader, optimizer_adv, epoch, epsilon = 0.1)
    print("\nTest on standard model:")
    test(model, device, test_loader)
    #print("Now with adversarial examples")
    #acc, ex = adv_test(model, device, test_loader, epsilon = 0.1)

    print("\nTest on adversarially trained model:")
    test(model_adv, device, test_loader)
    #print("Now with adversarial examples")
    #adv_acc, adv_ex = adv_test(model_adv, device, test_loader, epsilon = 0.1)

        #scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "/content/drive/My Drive/NNProj/FashionMNIST.pt")
        torch.save(model_adv.state_dict(), "/content/drive/My Drive/NNProj/FashionMNIST2_adv.pt")

In [0]:
#main()

In [0]:
# Initialize loaders and cuda
use_cuda = torch.cuda.is_available()
print("CUDA? " + str(use_cuda))

torch.manual_seed(20294798)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                    ])),
    batch_size=100, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                        transforms.ToTensor(),
                    ])),
    batch_size=1, shuffle=True, **kwargs)

test_loader_batch = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                        transforms.ToTensor(),
                    ])),
    batch_size=1000, shuffle=True, **kwargs)

In [0]:

pretrained_model = "/content/drive/My Drive/NNProj/MNIST.pt"


model = Net().to(device)
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))
model.eval()

In [0]:
# Generate Adversarial examples for analysis
num_examples = 1000
s = set()
from_to = {}
for d in range(8, 9):
    s.add(d)
    from_to[d] = None
adv_examples_list = torch.zeros([num_examples, 1, 28, 28])
examples_list = torch.zeros([num_examples, 1, 28, 28])
targets = torch.zeros(num_examples)
predictions = torch.zeros(num_examples)
#while len(s) != 0:
i = 0
while i < num_examples:
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    t = target.item()
    #if t in s:

    output = model(data)

    pred = output.argmax(dim=1, keepdim=True)
    if pred == target:
        adv_ex, success, _ = GA.gen_attack(10, data, target, 0.2, 0.15, 0.1, 5000, 2, model, GA.fitness, device)
        adv_output = model(adv_ex + data)
        adv_pred = adv_output.argmax(dim=1, keepdim=True)
        if adv_pred != target:
            #print(str(t) + " became " + str(adv_pred.item()))
            adv_examples_list[i] = adv_ex + data
            examples_list[i] = data
            targets[i] = t
            predictions[i] = adv_pred
            #from_to[t] = adv_pred.item()
            #s.remove(t)
            i += 1
            if i %100 == 0:
                print(i)





In [0]:
# Generate FGSM examples
examples_list.requires_grad = True

model.zero_grad()
output = model(examples_list.cuda())
loss = F.nll_loss(output, targets.long().cuda())
loss.backward()

# Generating Adversarial Examples
# Collect gradient of inputs
data_grad = examples_list.grad.data

# FGSM Attack
FGSM_examples = fgsm_attack(examples_list, 0.2, data_grad).cuda()
FGSM_examples.detach()
FGSM_output = model(FGSM_examples)
FGSM_pred = FGSM_output.argmax(dim=1, keepdim=True)

In [0]:
## Plot one of each examples with predicitons
fashion_dict = {0: 'tshirt', 1: 'trouser', 2: 'pullover', 3: 'dress', 4: 'coat', 5:'sandal', 6: 'shirt', 7: 'sneaker', 8: 'bag', 9: 'boot'}
fig, a = plt.subplots(1, 10, figsize=(20, 20))
for b in range(10):

    a[b].imshow(FGSM_examples[b].detach().cpu()[0], cmap="gray")
    a[b].axis('off')
    #a[b].set_title(FGSM_pred[b].item(), fontsize=20)
    #a[b].set_title(fashion_dict[from_to[b]], fontsize=20)
    #a[b].set_title(fashion_dict[b], fontsize=20)
    #a[b].set_title(from_to[b], fontsize=20)
    a[b].set_title(fashion_dict[FGSM_pred[b].item()], fontsize=20)

In [0]:
#Plot example and attacks
fig, a = plt.subplots(1, 3, figsize=(20, 20))
a[0].imshow(examples_list[8].detach().cpu()[0], cmap="gray")
a[0].axis('off')
a[0].set_title(fashion_dict[8], fontsize=40)
a[1].imshow(adv_examples_list[8].detach().cpu()[0], cmap="gray")
a[1].axis('off')
a[1].set_title(fashion_dict[from_to[8]], fontsize=40)
a[2].imshow(FGSM_examples[8].detach().cpu()[0], cmap="gray")
a[2].axis('off')
a[2].set_title(fashion_dict[FGSM_pred[8].item()], fontsize=40)
fig.show()